<a href="https://colab.research.google.com/github/hincz-lab/motion-blur-microscopy/blob/main/Analysis_Material/Count_Cells/Count_Cells_With_Size_Threshold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This code is used to count cells with only a phase one network, as well as a size threshold for classification. To use this code, you will need a trained phase one network, as well as images to be counted.

First, we will run the following block of code to clone a repository from github.

After cloning the repository, you can see the repository in the "Files" section of Colab, located on the left-hand side of the screen.

In [1]:
#@title Clone Github Repository
!git clone https://github.com/hincz-lab/motion-blur-microscopy.git

Cloning into 'motion-blur-microscopy'...
remote: Enumerating objects: 16748, done.
remote: Counting objects: 100% (4630/4630), done.
remote: Compressing objects: 100% (539/539), done.
remote: Total 16748 (delta 4181), reused 4496 (delta 4072), pack-reused 12118
Receiving objects: 100% (16748/16748), 120.12 MiB | 15.10 MiB/s, done.
Resolving deltas: 100% (7973/7973), done.
Updating files: 100% (162/162), done.


Next, let's import any packages or libraries we need for this document.

In [ ]:
#@title Import Packages And Libraries
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 as cv2
import numpy as np
from scipy.ndimage import label
from skimage import measure
import os as os

#Stable Versions:
# Pandas 1.4.4
# Tensorflow 2.11.0
# CV2 4.7.0
# Numoy 1.22.4
# Scipy 1.10.1
# skimage 0.19.3
# # Matplotlib 3.7.1

'3.7.1'

Now, we can complete our analysis. To begin, we will first upload the images we want to analyze. Upload any images you want to analyze to the directory motion-blur-microscopy -> Analysis_Material -> Count_Cells -> Images_For_Analysis. 

Furthermore, you can upload a suitable phase one segmentation network to the directory motion-blur-microscopy -> Analysis_Material -> Count_Cells -> Networks. Note, having more than two networks uploaded may cause unexpected results, as the code to follow will grab and use the first network it finds.

After you have done this, you can run the following block of code to generate counts of cells for each of the uploaded images using a size threshold for classification.

Note, you may need to open the code in the following block and change the *threshold* variable, which is the threshold used for classification by size.

In [ ]:
#@title Count Cells

base_Directory = "motion-blur-microscopy/Analysis_Material/Count_Cells/"

for subdirectory in os.listdir(base_Directory):
  if '.ipynb_checkpoints' in os.listdir(base_Directory  + subdirectory):
    os.rmdir(base_Directory + subdirectory + "/.ipynb_checkpoints")
  if 'blank.txt' in os.listdir(base_Directory  + subdirectory):
      os.remove(base_Directory + subdirectory + "/blank.txt")

output_Data_Frame = pd.DataFrame(columns = ["File Name", "Counts"])
channel_Directory = base_Directory + "/Images_For_Analysis/"
network_File_Path = base_Directory + "Networks/" + os.listdir(base_Directory + "Networks/")[0]

network = tf.keras.models.load_model(network_File_Path)

def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

threshold = [90]

for image_Name in os.listdir(channel_Directory):
    print("Analyzing " + image_Name[:-4])
    full_Channel = plt.imread(channel_Directory + image_Name)
    if np.max(full_Channel) == int(np.max(full_Channel)) and len(str(np.max(full_Channel))) == len(str(int(np.max(full_Channel)))):
        full_Channel = full_Channel/255.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel, cv2.COLOR_GRAY2RGB)
    if np.shape(full_Channel)[2] == 4:
        full_Channel = full_Channel.copy()[:,:,0:3]
    image_Height, image_Width, channels = np.shape(full_Channel)
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))
    x_Slider = 0
    y_Slider = 0
    output_Array = np.zeros((128,128))
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            output = network.predict(current_Tile_Normalized, verbose = 0)

            for i in range(128):
                for j in range(128):
                    output_Array[i,j] = np.argmax(output[0,i,j,:])
            
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    for i in range(image_Height_Resized):
        for j in range(image_Width_Resized):
            if output_Image[i,j] != 0:
                output_Image[i,j] = 1
            else:
                continue
    blobs, number_Of_Blobs = label(output_Image)
    properties = measure.regionprops(blobs)
    for thresh in threshold:
        centroids = [prop.centroid for prop in properties if prop.area > thresh]
        output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids)]], columns = ["File Name", "Counts"]))
    display(output_Data_Frame)
    print("======================================")

Analyzing Frame_8


<ipython-input-30-d5efae18b6a8>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids)]], columns = ["File Name", "Counts"]))


,File Name,Counts
0,Frame_8.png,37


Analyzing Frame_10


<ipython-input-30-d5efae18b6a8>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids)]], columns = ["File Name", "Counts"]))


...

<ipython-input-30-d5efae18b6a8>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids)]], columns = ["File Name", "Counts"]))


,File Name,Counts
0,Frame_8.png,37
0,Frame_10.png,33
0,Frame_5.png,38
0,Frame_6.png,38
0,Frame_9.png,35
0,Frame_3.png,37
0,Frame_1.png,37
0,Frame_4.png,35
0,Frame_2.png,37


Analyzing Frame_7


<ipython-input-30-d5efae18b6a8>:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids)]], columns = ["File Name", "Counts"]))


,File Name,Counts
0,Frame_8.png,37
0,Frame_10.png,33
0,Frame_5.png,38
0,Frame_6.png,38
0,Frame_9.png,35
0,Frame_3.png,37
0,Frame_1.png,37
0,Frame_4.png,35
0,Frame_2.png,37
0,Frame_7.png,39


Now, we can save the counts to an excel file, for use later. You can save your counts by running the following block of code. Make sure you download the .csv file that is written, as the Google Colab runtime will delete all files when closed.

The output file will be located at motion-blur-microscopy -> Analysis_Material -> Count_Cells -> Outputs

In [ ]:
#@title Save Counts
output_Data_Frame.to_csv(base_Directory + "/Outputs/Counts.csv")